# Query #1

``` mysql
SELECT
  o.service_id,
  o.user_segment,
  pd.fuel_id,
  COUNT(o.order_id) AS total_orders,
  COUNT(p.payment_item_id) AS total_payments,
  CAST(COUNT(p.payment_item_id) AS UNSIGNED) / COUNT(o.order_id) AS conversion,
  COALESCE(SUM(p.amount), 0) / (NULLIF(COUNT(DISTINCT o.order_id), 0)) AS ARPO
FROM
  orders o
  LEFT JOIN payments p ON o.order_id = p.order_id
  INNER JOIN proposal_detail pd ON o.proposal_id = pd.proposal_id
WHERE
  o.created_at >= (CURRENT_DATE - INTERVAL 30 DAY)
GROUP BY
  o.service_id, o.user_segment, pd.fuel_id;
```

## Query #2

``` mysql
SELECT
    o.user_id,
    SUM(p.amount) AS total_revenue
FROM
    orders o
JOIN
    payments p ON o.order_id = p.order_id
WHERE
    o.created_at >= CURDATE() - INTERVAL 30 DAY
GROUP BY
    o.user_id
ORDER BY
    total_revenue DESC
LIMIT 10;
```

## Query #3

``` mysql
WITH ConversionRates AS (
  SELECT
    pd.state_id,
    pd.transport_type_id,
    COUNT(p.payment_item_id) / COUNT(o.order_id) AS conversion_rate,
    ROW_NUMBER() OVER (PARTITION BY pd.state_id ORDER BY COUNT(p.payment_item_id) / COUNT(o.order_id) DESC) AS rn
  FROM
    proposal_detail pd
  JOIN
    orders o ON pd.proposal_id = o.proposal_id
  LEFT JOIN
    payments p ON o.order_id = p.order_id AND p.created_at BETWEEN o.created_at AND o.created_at + INTERVAL 30 DAY
  WHERE
    o.created_at >= CURDATE() - INTERVAL 30 DAY
  GROUP BY
    pd.state_id, pd.transport_type_id
)
SELECT
  state_id,
  transport_type_id,
  conversion_rate
FROM
  ConversionRates
WHERE
  rn = 1;

```

## Query #4

``` mysql
WITH MarkaSum AS (
    SELECT
        pd.state_id,
        mm.marka,
        SUM(p.amount) AS total_amount
    FROM
        payments p
    JOIN
        orders o ON p.order_id = o.order_id
    JOIN
        proposal_detail pd ON o.proposal_id = pd.proposal_id
    JOIN
        marksModels mm ON pd.proposal_id = mm.proposal_id
    GROUP BY
        pd.state_id, mm.marka
),
RankedMarka AS (
    SELECT
        state_id,
        marka,
        total_amount,
        RANK() OVER (PARTITION BY state_id ORDER BY total_amount DESC) AS rank_marka
    FROM
        MarkaSum
),
TopModels AS (
    SELECT
        pd.state_id,
        mm.marka,
        mm.model,
        SUM(p.amount) AS model_amount,
        RANK() OVER (PARTITION BY pd.state_id, mm.marka ORDER BY SUM(p.amount) DESC) AS model_rank
    FROM
        payments p
    JOIN
        orders o ON p.order_id = o.order_id
    JOIN
        proposal_detail pd ON o.proposal_id = pd.proposal_id
    JOIN
        marksModels mm ON pd.proposal_id = mm.proposal_id
    GROUP BY
        pd.state_id, mm.marka, mm.model
)
SELECT
    tm.state_id,
    tm.marka,
    tm.model,
    tm.model_amount
FROM
    TopModels tm
JOIN
    RankedMarka rm ON tm.state_id = rm.state_id AND tm.marka = rm.marka
WHERE
    rm.rank_marka = 1 AND tm.model_rank <= 3
ORDER BY
    tm.state_id, tm.marka, tm.model_rank;

```